In [ ]:
pip install -U vllm

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    dtype="float16",
    gpu_memory_utilization=0.85,
    enforce_eager = True,
    trust_remote_code=True,
    max_model_len=4096,
)

In [ ]:
sampling_params = SamplingParams(
    n=3,
    temperature=0.6,
    top_p=0.95,
    max_tokens=4096,
)

In [ ]:
import json

records = []
with open("/content/FinalDataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from itertools import islice
from tqdm.auto import tqdm

In [ ]:
out_f = open("cots1.jsonl", "a", encoding="utf-8")
base = "https://huggingface.co/datasets/uzaymacar/math-rollouts/resolve/main"
# root = f"{base}/deepseek-r1-distill-qwen-14b/temperature_0.6_top_p_0.95/incorrect_base_solution"
root = f"{base}/deepseek-r1-distill-qwen-14b/temperature_0.6_top_p_0.95/correct_base_solution"
dataset = load_dataset(
    "json",
    data_files=f"{root}/problem_*/problem.json",
    streaming=True
)

start_idx = 0
num_examples = 20


example_iterator = islice(dataset["train"], start_idx, start_idx + num_examples)
for i, ex in enumerate(
    tqdm(example_iterator, total=num_examples, desc="Generating Answers"),
    start=start_idx
):
# for i, ex in enumerate(
#     islice(dataset["train"], start_idx, start_idx + num_examples),
#     start=start_idx
# ):
    problem = ex["problem"]
    prompt = ["<|User|>" + problem+ "<|Assistant|><think>"]
    outputs = llm.generate(prompt, sampling_params)
    answer_text = outputs[0].outputs[0].text

    record = {
        "q_id": i,
        "problem": problem,
        "answer": answer_text
    }

    out_f.write(json.dumps(record, ensure_ascii=False) + "\n")
    out_f.flush()

out_f.close()